# Email Assistant Agent - Development Notebook

This notebook is for developing and testing the email assistant agent components.

## Setup and Configuration

In [ ]:
# Install required packages (run once)
# !pip install -r requirements.txt

In [ ]:
import os
from dotenv import load_dotenv
from email_agent import EmailAgent
from evaluate_triage import TriageEvaluator

# Load environment variables
load_dotenv()

# Check if API key is set
if not os.getenv("GOOGLE_API_KEY"):
    print("⚠️  Please set your GOOGLE_API_KEY in the .env file")
else:
    print("✅ Google API key is configured")

## Test Individual Email Processing

In [ ]:
# Initialize the agent
agent = EmailAgent()

# Test with a sample email
test_email = {
    "content": "Hi, I'd like to schedule a meeting next week to discuss the project proposal. Are you available Tuesday afternoon?",
    "sender": "client@business.com",
    "subject": "Meeting Request - Project Discussion"
}

result = agent.process_email(
    email_content=test_email['content'],
    sender=test_email['sender'],
    subject=test_email['subject']
)

print(f"Triage Decision: {result.get('triage_decision')}")
print(f"Reasoning: {result.get('reasoning')}")
print("\nAgent Messages:")
for msg in result.get('messages', []):
    print(f"- {msg.get('content', '')[:100]}...")

## Run Triage Evaluation

In [ ]:
# Run the full triage evaluation
evaluator = TriageEvaluator()
results = evaluator.evaluate_triage_accuracy()

## Interactive Testing

In [ ]:
# Interactive email testing
def test_custom_email():
    print("Enter email details (press Enter twice to finish content):")
    
    sender = input("From: ")
    subject = input("Subject: ")
    
    print("Content (press Enter twice when done):")
    content_lines = []
    while True:
        line = input()
        if line == "" and content_lines and content_lines[-1] == "":
            break
        content_lines.append(line)
    
    content = "\n".join(content_lines[:-1])  # Remove the last empty line
    
    result = agent.process_email(
        email_content=content,
        sender=sender,
        subject=subject
    )
    
    print("\n" + "="*50)
    print(f"Triage Decision: {result.get('triage_decision')}")
    print(f"Reasoning: {result.get('reasoning')}")
    
    return result

# Uncomment to run interactive testing
# test_custom_email()

## Analyze Results

In [ ]:
import json
import matplotlib.pyplot as plt

# Load evaluation results if they exist
try:
    with open('triage_evaluation_results.json', 'r') as f:
        eval_results = json.load(f)
    
    # Create accuracy visualization
    categories = ['ignore', 'notify_human', 'respond']
    accuracies = []
    
    for category in categories:
        category_tests = [r for r in eval_results['detailed_results'] if r['expected'] == category]
        category_correct = [r for r in category_tests if r['correct']]
        if category_tests:
            accuracy = len(category_correct) / len(category_tests)
            accuracies.append(accuracy)
        else:
            accuracies.append(0)
    
    plt.figure(figsize=(10, 6))
    plt.bar(categories, accuracies, color=['red', 'orange', 'green'], alpha=0.7)
    plt.axhline(y=0.8, color='blue', linestyle='--', label='Target (80%)')
    plt.ylabel('Accuracy')
    plt.title('Triage Accuracy by Category')
    plt.ylim(0, 1)
    plt.legend()
    
    for i, v in enumerate(accuracies):
        plt.text(i, v + 0.02, f'{v:.2%}', ha='center')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Overall Accuracy: {eval_results['accuracy']:.2%}")
    print(f"Target Met: {'✅' if eval_results['passed_criteria'] else '❌'}")
    
except FileNotFoundError:
    print("No evaluation results found. Run the evaluation first.")